# Installation

In [ ]:
# 1. Create and activate a conda env with R dependencies (for full functionality):
conda create -n scFates -c conda-forge -c r python=3.11 r-mgcv rpy2 -y
conda activate scFates

# 2. Install scFates:
pip install scFates

# (Optional) If you want the very latest development version:
pip install git+https://github.com/LouisFaure/scFates


# Import Libraries

In [1]:
import scanpy as sc
import scFates as scf
import matplotlib.pyplot as plt

# Data Acquisition

In [2]:
!pwd

/storage/users/job37yv/Projects/Teaching/scFates


In [1]:
# 1. Make a directory and enter it
!mkdir -p data/E-MTAB-6149/fastq
!cd data/E-MTAB-6149/fastq

In [4]:
from ftplib import FTP
import os

# 1. Make sure output dir exists and switch into it
out_dir = 'data/E-MTAB-6149/fastq'
os.makedirs(out_dir, exist_ok=True)
os.chdir(out_dir)

# 2. Connect & login
ftp = FTP('ftp.ebi.ac.uk')
ftp.login()  # anonymous

# 3. Navigate to the folder containing your files
ftp.cwd('biostudies/fire/E-MTAB-/149/E-MTAB-6149/Files')

# 4. List of files to grab
files = [
    "BT1A.R1.fastq.gz",
    "BT1A.R2.fastq.gz",
    "1247.R1.fastq.gz",
    "1247.R2.fastq.gz",
    "1247.R3.fastq.gz"
]

# 5. Download each
for fn in files:
    print(f"Downloading {fn} …")
    with open(fn, 'wb') as f:
        ftp.retrbinary(f'RETR {fn}', f.write)

ftp.quit()
print("Done.")


Done.


In [4]:
!unzip E-MTAB-6149.processed.2.zip -d E-MTAB-6149-data

unzip:  cannot find or open E-MTAB-6149.processed.2.zip, E-MTAB-6149.processed.2.zip.zip or E-MTAB-6149.processed.2.zip.ZIP.


In [ ]:
adata = sc.read_csv('E-MTAB-6149-data/counts_matrix.csv.gz').T  # transpose if needed


# Metadata

In [13]:
import os
import pandas as pd

# 1. Your absolute “project” root (just up to scFates)
root_dir = "/home/job37yv/Projects_shared/Teaching/scFates"

# 2. Your data/output directory *relative* to that root
data_dir = "data/E-MTAB-6149/fastq"

# 3. Build the full path and make sure it exists
out_dir = os.path.join(root_dir, data_dir)
os.makedirs(out_dir, exist_ok=True)

# 4. (Optionally) cd into the project root if you want all operations relative to it
os.chdir(root_dir)


In [15]:
# 5. Prepare your metadata DataFrame
metadata = pd.DataFrame([
    {'sample':'BT1A',
     'organism':'Homo sapiens','
     individual':1,
     'disease':'lung carcinoma',
     'biopsy_site':'tumor core',
     'organism_part':'lung',
     'age':70,
     'age_unit':'year',
     'sex':'female'},
    {'sample':'BT1B',
     'organism':'Homo sapiens',
     'individual':1,
     'disease':'lung carcinoma',
     'biopsy_site':'tumor middle',
     'organism_part':'lung',
     'age':70,
     'age_unit':'year',
     'sex':'female'},
    {'sample':'BT1C',
     'organism':'Homo sapiens',
     'individual':1,
     'disease':'lung carcinoma',
     'biopsy_site':'tumor edge',
     'organism_part':'lung',
     'age':70,
     'age_unit':'year',
     'sex':'female'},
    {'sample':'BT2A',
     'organism':'Homo sapiens',
     'individual':2,
     'disease':'lung carcinoma',
     'biopsy_site':'tumor middle',
     'organism_part':'lung',
     'age':86,
     'age_unit':'year',
     'sex':'male'}
])

# 6. Write the CSV into your data folder
csv_path = os.path.join(out_dir, 'metadata.csv')
metadata.to_csv(csv_path, index=False)

print("Working dir:", os.getcwd())
print("Metadata written to:", csv_path)

Working dir: /storage/users/job37yv/Projects/Teaching/scFates
Metadata written to: /home/job37yv/Projects_shared/Teaching/scFates/data/E-MTAB-6149/fastq/metadata.csv


# Read mapping

In [6]:
from pathlib import Path
import stat

# Use current notebook directory as project root
root_dir = Path('.')
data_dir = root_dir / "data" / "E-MTAB-6149"
fastq_dir = data_dir / "fastq"
reference_dir = root_dir / "data" / "reference" / "GRCh38"
star_index_dir = reference_dir / "STAR_index"
results_dir = root_dir / "results" / "STARsolo" / "BT1A"
scripts_dir = root_dir / "scripts"

# Ensure directories exist (relative to project root)
for d in [data_dir, fastq_dir, reference_dir, star_index_dir, results_dir, scripts_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Script file paths
download_script = scripts_dir / "download_reference.sh"
index_script = scripts_dir / "build_star_index.sh"
run_script = scripts_dir / "run_star_solo.sh"

# 1) download_reference.sh
download_script.write_text(f"""#!/usr/bin/env bash
set -euo pipefail

ROOT="$(pwd)"
REF_DIR="$ROOT/data/reference/GRCh38"
mkdir -p "$REF_DIR"

echo "Downloading genome FASTA..."
wget -c https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/GRCh38.primary_assembly.genome.fa.gz \\
     -O "$REF_DIR/genome.fa.gz"
echo "Decompressing FASTA..."
gunzip -c "$REF_DIR/genome.fa.gz" > "$REF_DIR/GRCh38.primary_assembly.genome.fa"

echo "Downloading GTF..."
wget -c https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/gencode.v42.annotation.gtf.gz \\
     -O "$REF_DIR/annotations.gtf.gz"
echo "Decompressing GTF..."
gunzip -c "$REF_DIR/annotations.gtf.gz" > "$REF_DIR/gencode.v42.annotation.gtf"

echo "Reference download complete."
""")

# 2) build_star_index.sh
index_script.write_text(f"""#!/usr/bin/env bash
set -euo pipefail

ROOT="$(pwd)"
STAR_INDEX="$ROOT/data/reference/GRCh38/STAR_index"
mkdir -p "$STAR_INDEX"

echo "Building STAR index..."
STAR --runMode genomeGenerate \\
     --runThreadN 8 \\
     --genomeDir "$STAR_INDEX" \\
     --genomeFastaFiles "$ROOT/data/reference/GRCh38/GRCh38.primary_assembly.genome.fa" \\
     --sjdbGTFfile "$ROOT/data/reference/GRCh38/gencode.v42.annotation.gtf" \\
     --sjdbOverhang 100

echo "STAR index built at $STAR_INDEX"
""")

# 3) run_star_solo.sh
run_script.write_text(f"""#!/usr/bin/env bash
set -euo pipefail

# ensure STAR in PATH
export PATH="/home/job37yv/miniforge3/envs/scfates-star/bin:$PATH"

ROOT="$(pwd)"
FASTQ_DIR="$ROOT/data/E-MTAB-6149/fastq"
STAR_INDEX="$ROOT/data/reference/GRCh38/STAR_index"
OUT_DIR="$ROOT/results/STARsolo/BT1A"
WHITELIST="$CONDA_PREFIX/share/STAR/whitelist/3M-february-2018.txt"

mkdir -p "$OUT_DIR"
echo "Running STARsolo with 10× V2 parameters and whitelist $WHITELIST…"

STAR --runThreadN 8 \
     --genomeDir "$STAR_INDEX" \
     --readFilesIn \
         "$FASTQ_DIR/BT1A.R2.fastq.gz" \
         "$FASTQ_DIR/BT1A.R1.fastq.gz" \
     --readFilesCommand zcat \
     --outFileNamePrefix "$OUT_DIR/" \
     --soloType CB_UMI_Simple \
     --soloCBwhitelist "$WHITELIST" \
     --soloCBlen       14,0 \
     --soloUMIlen      10,0 \
     --soloCBstart     1,0 \
     --soloUMIstart    15,0 \
     --soloFeatures    Gene

echo "Done. STARsolo output in $OUT_DIR/Solo.out/Gene"
""")

# Make scripts executable
for script in [download_script, index_script, run_script]:
    script.chmod(script.stat().st_mode | stat.S_IXUSR)

# Inform user
print("Scripts created in './scripts':")
print("1) download_reference.sh")
print("2) build_star_index.sh")
print("3) run_star_solo.sh")
print("\nRun them in this order from your notebook root:")
print("bash scripts/download_reference.sh")
print("bash scripts/build_star_index.sh")
print("bash scripts/run_star_solo.sh")


Scripts created in './scripts':
1) download_reference.sh
2) build_star_index.sh
3) run_star_solo.sh

Run them in this order from your notebook root:
bash scripts/download_reference.sh
bash scripts/build_star_index.sh
bash scripts/run_star_solo.sh


In [3]:
!pwd

/storage/users/job37yv/Projects/Teaching/scFates


In [4]:
!bash scripts/download_reference.sh

--2025-05-25 19:30:23--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/GRCh38.primary_assembly.genome.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Decompressing FASTA...
--2025-05-25 19:30:42--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/gencode.v42.annotation.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Decompressing GTF...
Reference download complete.


In [14]:
bash scripts/build_star_index.sh

Building STAR index...
scripts/build_star_index.sh: line 9: STAR: command not found


In [15]:
bash scripts/run_star_solo.sh

Running STARsolo...
scripts/run_star_solo.sh: line 11: STAR: command not found


# Read counting

# Load mapped fastq into an AnnData

In [ ]:
import scanpy as sc

adata = sc.read_10x_mtx(
    'results/STARsolo/BT1A/Solo.out/Gene',     # path to matrix folder
    var_names='gene_symbols',
    cache=True
)

# Add sample metadata
import pandas as pd
meta = pd.read_csv('data/E-MTAB-6149/fastq/metadata.csv')
# filter metadata to BT1A row
bt1a_meta = meta[meta.sample == 'BT1A'].iloc[0].to_dict()
for k, v in bt1a_meta.items():
    adata.obs[k] = v

# QC and inspect
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
print(adata)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0.4)


# Preprocessing

In [ ]:
# Basic QC
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Normalization & log-transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Identify highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]

# Scale and PCA
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

# Neighbors & UMAP
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
sc.tl.umap(adata)


# Trajectory Inference with scFates

In [ ]:
# 1. Compute Diffusion Map (optional but recommended)
sf.pp.diffusion(adata)

# 2. Fit a principal tree
sf.tl.tree(adata, use_pca=True)  

# 3. Compute pseudotime along the tree
sf.tl.pseudotime(adata, root_cell='0')  # choose an appropriate root

# 4. Assign “fates” (branches)
sf.tl.fates(adata)


# Visualization

In [ ]:
# UMAP colored by pseudotime
sc.pl.umap(adata, color='pseudotime', cmap='viridis')

# Overlay tree on UMAP
sf.pl.tree_graph(adata, basis='umap', edge_width=1.0)

# Plot individual branch trajectories
sf.pl.plot_branch(adata, branch='branch_1', keys=['GeneA','GeneB'])


# Differential Gene Testing Along Pseudotime

In [ ]:
# Test association of gene expression with pseudotime
sf.tl.test_association(adata, key='pseudotime')

# Extract top genes
top = adata.uns['test_association']['scores'].sort_values('pval').head(10)
print(top)

# Heatmap of top dynamic genes
sf.pl.heatmap(adata, var_names=list(top.index), sort_by='pseudotime')
